In [1]:
from glob import glob
from caiman_online.utils import make_ain
from caiman_online.pipelines import OnAcidPipeline, SeededPipeline
import logging

LOGFILE = 'E:/caiman_scratch/ori3/caiman/out/pipeline_test.log'
LOGFORMAT = '{relativeCreated:08.0f} - {levelname:8} - [{module}:{funcName}:{lineno}] - {message}'
logging.basicConfig(level=logging.ERROR, format=LOGFORMAT, style='{')
logger = logging.getLogger('caiman_online')
logger.setLevel(logging.DEBUG)

folder =  'E:/caiman_scratch/ori3'
mm3d_path = glob('E:/caiman_scratch/ori3/*.mat')[0]
nchannels = 2
nplanes = 3
xslice = slice(100,400)
batch_size_tiffs = 50

# motion correction and CNMF
dxy = (1.5, 1.5) # spatial resolution in x and y in (um per pixel)
max_shift_um = (15., 15.) # maximum shift in um
patch_motion_xy = (100., 100.) # patch size for non-rigid correction in um

params = {
    # CNMF
    'fr': 6.36,
    'p': 1,  # deconv 0 is off, 1 is slow, 2 is fast
    'nb': 3,  # background compenents -> nb: 3 for complex
    'decay_time': 1.0,  # sensor tau
    'gSig': (7, 7),  # expected half size of neurons in pixels, very important for proper component detection
    'only_init': False,  # has to be `False` when seeded CNMF is used
    'rf': None,  # half-size of the patches in pixels. Should be `None` when seeded CNMF is used.
    'ssub': 1,
    'tsub': 1,
    'do_merge': False, # new found param, testing
    'update_background_components': True,
    'merge_thr': 0.999,
    # 'optimize_g': True,
    
    # motion
    'gSig_filt': (7, 7), # high pass spatial filter for motion correction
    'nonneg_movie': True,
    'niter_rig': 1,
    'pw_rigid': False,  # piece-wise rigid flag, slower
    'max_deviation_rigid': 3,
    'overlaps': (24, 24),
    'max_shifts': [int(a/b) for a, b in zip(max_shift_um, dxy)],
    'strides': tuple([int(a/b) for a, b in zip(patch_motion_xy, dxy)]),
    'num_frames_split': 50,
    'border_nan': 'copy',
    
    # online
    'init_method': 'seeded',
    'epochs': 2,
    'show_movie': False,
    'motion_correct': True,
    'expected_comps': 300,
    'update_num_comps':False,
    'max_num_added':0,
}

class TestSeededPipeline(SeededPipeline):
    def get_tiffs(self):
        all_tiffs = list(self.folder.glob('*.tif*'))
        chunked = [all_tiffs[i:i + self.batch_size_tiffs] for i in range(0, len(all_tiffs), self.batch_size_tiffs)]
        tiffs = chunked[self.iters]
        splits = self.validate_tiffs(tiffs)
        logger.debug(f'Processing files: {tiffs}')
        return tiffs, splits
    
class TestOnAcidPipeline(OnAcidPipeline):
    def get_tiffs(self):
        all_tiffs = list(self.folder.glob('*.tif*'))
        chunked = [all_tiffs[i:i + self.batch_size_tiffs] for i in range(0, len(all_tiffs), self.batch_size_tiffs)]
        tiffs = chunked[self.iters]
        splits = self.validate_tiffs(tiffs)
        logger.debug(f'Processing files: {tiffs}')
        return tiffs, splits

def test_seeded():
    logger.info('Running test of seeded caiman batch pipeline.')
    Ain = [make_ain(mm3d_path, p, 100, 400) for p in range(nplanes)]
    seeded = TestSeededPipeline(folder, params, nchannels, nplanes, 
                            x_start=100, x_end=400, Ain=Ain, batch_size_tiffs=batch_size_tiffs)
    
    ntiffs = len(glob(folder + '/*.tif*'))
    rounds = ntiffs//batch_size_tiffs
    
    for _ in range(rounds):
        seeded.fit_batch()
        
def test_online():
    logger.info('Running test of seeded OnACID pipeline.')
    Ain = [make_ain(mm3d_path, p, 100, 400) for p in range(nplanes)]
    seeded = TestOnAcidPipeline(folder, params, nchannels, nplanes, 
                            x_start=100, x_end=400, Ain=Ain, batch_size_tiffs=batch_size_tiffs)
    
    ntiffs = len(glob(folder + '/*.tif*'))
    rounds = ntiffs//batch_size_tiffs
    
    for _ in range(rounds):
        seeded.fit_batch()

In [2]:
test_online()

00041474 - INFO     - [<ipython-input-1-f2de7cc5bed7>:test_online:93] - Running test of seeded OnACID pipeline.


Plane 0: Found 82 sources from MM3D...
Plane 1: Found 122 sources from MM3D...
Plane 2: Found 94 sources from MM3D...


00044949 - DEBUG    - [<ipython-input-1-f2de7cc5bed7>:get_tiffs:77] - Processing files: [WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00002.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00003.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00004.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00005.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00006.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00007.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00008.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00009.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00010.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00011.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00012.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00013.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00014.tif'), WindowsPath('E:/caiman_scratch/ori3/2020

***** Starting Plane 0 OnACID... *****
Starting OnACID processing...


00045082 - DEBUG    - [workers:_start_cluster:93] - Local cluster ready.
00050798 - INFO     - [wrappers:wrapper_timer:17] - <caiman_online.workers.make_tiff> done in 5.715s
C:\Users\Will\Anaconda3\envs\caiman-online\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 10 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
00112286 - INFO     - [wrappers:wrapper_timer:17] - <caiman_online.workers.do_fit> done in 61.486s
00112340 - DEBUG    - [pipelines:fit_batch:329] - Saving data.
00113707 - ERROR    - [workers:cleanup_tmp:159] - Unable to remove file: E:\caiman_scratch\ori3\caiman\tmp\onlinemovie.tif
00113708 - DEBUG    - [pipelines:fit_batch:317] - Starting OnACID worker.
00113709 - DEBUG    - [workers:__init__:46] - Setting up params...
00113764 - DEBUG    - [workers:_setup_folders:107] - Set temp_path to E:\caiman_scratch\ori3\caiman\tmp
00113765 - DEBUG    - [workers:_setup_folder

...

***** Plane 2 done. 71.4169 s *****


00265124 - DEBUG    - [pipelines:fit_batch:380] - Successfully saved all_data to file.
00265126 - DEBUG    - [workers:_stop_cluster:98] - Cluster stopped.
00265126 - DEBUG    - [workers:__del__:76] - Worker object destroyed on delete.
00272325 - DEBUG    - [<ipython-input-1-f2de7cc5bed7>:get_tiffs:77] - Processing files: [WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00052.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00053.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00054.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00055.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00056.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00057.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00058.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00059.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00060.tif'), WindowsPath('E:/caiman_scratch/ori3/20200902_w18_3_e3_00061.tif'), Window

***** Starting Plane 0 OnACID... *****
Starting OnACID processing...


00272467 - DEBUG    - [workers:_start_cluster:93] - Local cluster ready.


KeyboardInterrupt: 

In [5]:
numpy.__config__.show()

blas_mkl_info:
    libraries = ['blas', 'cblas', 'lapack', 'blas', 'cblas', 'lapack']
    library_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\include']
blas_opt_info:
    libraries = ['blas', 'cblas', 'lapack', 'blas', 'cblas', 'lapack', 'blas', 'cblas', 'lapack']
    library_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\include']
lapack_mkl_info:
    libraries = ['blas', 'cblas', 'lapack', 'blas', 'cblas', 'lapack']
    library_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['D:\\bld\\numpy_1595523078866\\_h_env\\Library\\include']
lapack_opt_info:
    libraries = ['blas', 'cblas', 'lapack'

In [4]:
import numpy